In [ ]:
%matplotlib inline
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA


In [ ]:
#carrega o dataset
dados = pd.read_csv('/home/erasmor/Downloads/2017/todos_apenas_baixa_representatividade.csv',sep=",",encoding = 'utf-8',  header=0,na_values='.',dtype={'Label':'category'})
#remove valores infinitos
dados.replace(-np.Inf, np.nan)
#substitui valores NaN
dados.fillna(dados.mean())


In [ ]:
dados.dtypes

In [ ]:
dados.memory_usage(deep=True)

In [ ]:
# verifica quantas instâncias (linhas) e quantos atributos (colunas) a base de dados contém
print("numero de linhas e colunas: ",dados.shape)

In [ ]:
#visualizar distribições por classes contidas no csv - informar nome da classe alvo
print(dados.groupby('Label').size())

In [ ]:
cols = list(dados.columns)
# colunas que nao serao normalizadas
cols.remove('Label')
# Copiando os dados e aplicando a normalizacao por reescala nas colunas do DataFrame que contem
# valores continuos. Por padrao, o metodo minmax_scale reescala com min=0 e max=1.
dados = dados[~dados.isin([np.nan, np.inf, -np.inf]).any(1)]
dados[cols] = dados[cols].apply(minmax_scale)

In [ ]:
#define as colunas de atributos e a coluna da classe (de 0 a 78 são atributos e após a 78 é a classe)
X_raw = dados.iloc[:, :-1].values # atributos
y_raw = dados.iloc[:, 78].values # classe de ataques
X_raw = np.nan_to_num(X_raw.astype(np.float32))

In [ ]:
#transformar a variável y com valores categóricos (classses de ataques) em valores numéricos:
labelencoder_y = LabelEncoder()
y_raw = labelencoder_y.fit_transform(y_raw)

In [ ]:
# Instanciando um PCA. O parametro n_components indica a quantidade de dimensoes que a base
# original sera reduzida.
pca = PCA(n_components=10, whiten=True,random_state=42)

In [ ]:
# Aplicando o pca na base de dados. O atributo 'values' retorna um numpy.array
# de duas dimensões (matriz) contendo apenas os valores numericos do DataFrame.
X_raw = pca.fit_transform(X_raw)

In [ ]:
def uncertain_sampling_knn(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
           
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    # X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw)) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (bruto) de acordo com a dobra em uso
    X_treino, y_treino = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos para treinamento inicial
    n_labeled_examples = X_treino.shape[0]
    training_indices = np.random.randint(low=0, high=n_labeled_examples, size=5)
    
    X_train = X_treino[training_indices]
    y_train = y_treino[training_indices]
    
    #Isole os exemplos do pool para consultar
    
    X_pool = np.delete(X_treino, training_indices, axis=0)
    y_pool = np.delete(y_treino, training_indices, axis=0)
   
    #instanciando classificadores de aprendizado ativo
    learner_knn = KNeighborsClassifier(n_neighbors=5)
    arquivo_performance_knn = open("random_performance_knn_dobra_"+indica_pool+".txt","a")
    arquivo_history_knn = ("random_history_knn_dobra_"+indica_pool+".csv")
        
    for index in range(N_QUERIES):
        n_labeled_examples_pool = X_pool.shape[0]
        training_indices_pool = np.random.randint(low=0, high=n_labeled_examples_pool, size=BATCH_SIZE)
        X_temp= X_pool[training_indices_pool]
        y_temp= y_pool[training_indices_pool]
        X_train=np.append(X_train,X_temp, axis=0)
        y_train=np.append(y_train,y_temp, axis=0)
        learner_knn.fit(X_train, y_train)
        predictions = learner_knn.predict(X_teste)
        clear_output(wait=True)
        performance_history_knn.append(accuracy_score(y_teste, predictions))
        print ('Accuracy_knn after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
        arquivo_performance_knn.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_performance_knn.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_performance_knn.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_performance.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_performance_knn.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
        #print ("========================================")
        arquivo_performance_knn.write('======================================== \n')
        #apaga registros consultados
        X_pool = np.delete(X_pool, training_indices_pool, axis=0)
        y_pool = np.delete(y_pool, training_indices_pool, axis=0)
        
    arquivo_performance_knn.write("\n Avaliação_final_knn \n")
    arquivo_performance_knn.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_history_knn, performance_history_knn,delimiter=",")
    arquivo_performance_knn.close()
    
def uncertain_sampling_rf(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
           
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    # X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw)) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (bruto) de acordo com a dobra em uso
    X_treino, y_treino = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos para treinamento inicial
    n_labeled_examples = X_treino.shape[0]
    training_indices = np.random.randint(low=0, high=n_labeled_examples, size=5)
    
    X_train = X_treino[training_indices]
    y_train = y_treino[training_indices]
    
    #Isole os exemplos do pool para consultar
    
    X_pool = np.delete(X_treino, training_indices, axis=0)
    y_pool = np.delete(y_treino, training_indices, axis=0)
   
    #instanciando classificadores de aprendizado ativo
    learner_rf = RandomForestClassifier(random_state=42)
    arquivo_performance_rf = open("random_performance_rf_dobra_"+indica_pool+".txt","a")
    arquivo_history_rf = ("random_history_rf_dobra_"+indica_pool+".csv")
        
    for index in range(N_QUERIES):
        n_labeled_examples_pool = X_pool.shape[0]
        training_indices_pool = np.random.randint(low=0, high=n_labeled_examples_pool, size=BATCH_SIZE)
        X_temp= X_pool[training_indices_pool]
        y_temp= y_pool[training_indices_pool]
        X_train=np.append(X_train,X_temp, axis=0)
        y_train=np.append(y_train,y_temp, axis=0)
        learner_rf.fit(X_train, y_train)
        predictions = learner_rf.predict(X_teste)
        clear_output(wait=True)
        performance_history_rf.append(accuracy_score(y_teste, predictions))
        print ('Accuracy_rf after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
        arquivo_performance_rf.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_performance_rf.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_performance_rf.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_performance.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_performance_rf.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
        #print ("========================================")
        arquivo_performance_rf.write('======================================== \n')
        #apaga registros consultados
        X_pool = np.delete(X_pool, training_indices_pool, axis=0)
        y_pool = np.delete(y_pool, training_indices_pool, axis=0)
        
    arquivo_performance_rf.write("\n Avaliação_final_rf \n")
    arquivo_performance_rf.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_history_rf, performance_history_rf,delimiter=",")
    arquivo_performance_rf.close()

def uncertain_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
           
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    # X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw)) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (bruto) de acordo com a dobra em uso
    X_treino, y_treino = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos para treinamento inicial
    n_labeled_examples = X_treino.shape[0]
    training_indices = np.random.randint(low=0, high=n_labeled_examples, size=5)
    
    X_train = X_treino[training_indices]
    y_train = y_treino[training_indices]
    
    #Isole os exemplos do pool para consultar
    
    X_pool = np.delete(X_treino, training_indices, axis=0)
    y_pool = np.delete(y_treino, training_indices, axis=0)
   
    #instanciando classificadores de aprendizado ativo
    learner_tree = DecisionTreeClassifier(max_depth=5,min_samples_split=2,min_samples_leaf=2)
    arquivo_performance_tree = open("random_performance_tree_dobra_"+indica_pool+".txt","a")
    arquivo_history_tree = ("random_history_tree_dobra_"+indica_pool+".csv")
        
    for index in range(N_QUERIES):
        n_labeled_examples_pool = X_pool.shape[0]
        training_indices_pool = np.random.randint(low=0, high=n_labeled_examples_pool, size=BATCH_SIZE)
        X_temp= X_pool[training_indices_pool]
        y_temp= y_pool[training_indices_pool]
        X_train=np.append(X_train,X_temp, axis=0)
        y_train=np.append(y_train,y_temp, axis=0)
        learner_tree.fit(X_train, y_train)
        predictions = learner_tree.predict(X_teste)
        clear_output(wait=True)
        performance_history_tree.append(accuracy_score(y_teste, predictions))
        print ('Accuracy_tree after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
        arquivo_performance_tree.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_performance_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_performance_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_performance.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_performance_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
        #print ("========================================")
        arquivo_performance_tree.write('======================================== \n')
        #apaga registros consultados
        X_pool = np.delete(X_pool, training_indices_pool, axis=0)
        y_pool = np.delete(y_pool, training_indices_pool, axis=0)
        
    arquivo_performance_tree.write("\n Avaliação_final_tree \n")
    arquivo_performance_tree.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_history_tree, performance_history_tree,delimiter=",")
    arquivo_performance_tree.close()

def uncertain_sampling_mlp(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
           
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    # X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw)) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (bruto) de acordo com a dobra em uso
    X_treino, y_treino = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos para treinamento inicial
    n_labeled_examples = X_treino.shape[0]
    training_indices = np.random.randint(low=0, high=n_labeled_examples, size=5)
    
    X_train = X_treino[training_indices]
    y_train = y_treino[training_indices]
    
    #Isole os exemplos do pool para consultar
    
    X_pool = np.delete(X_treino, training_indices, axis=0)
    y_pool = np.delete(y_treino, training_indices, axis=0)
   
    #instanciando classificadores de aprendizado ativo
    learner_mlp = MLPClassifier(max_iter = 2000)
    arquivo_performance_mlp = open("random_performance_mlp_dobra_"+indica_pool+".txt","a")
    arquivo_history_mlp = ("random_history_mlp_dobra_"+indica_pool+".csv")
        
    for index in range(N_QUERIES):
        n_labeled_examples_pool = X_pool.shape[0]
        training_indices_pool = np.random.randint(low=0, high=n_labeled_examples_pool, size=BATCH_SIZE)
        X_temp= X_pool[training_indices_pool]
        y_temp= y_pool[training_indices_pool]
        X_train=np.append(X_train,X_temp, axis=0)
        y_train=np.append(y_train,y_temp, axis=0)
        learner_mlp.fit(X_train, y_train)
        predictions = learner_mlp.predict(X_teste)
        clear_output(wait=True)
        performance_history_mlp.append(accuracy_score(y_teste, predictions))
        print ('Accuracy_mlp after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
        arquivo_performance_mlp.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_performance_mlp.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_performance_mlp.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_performance.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_performance_mlp.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
        #print ("========================================")
        arquivo_performance_mlp.write('======================================== \n')
        #apaga registros consultados
        X_pool = np.delete(X_pool, training_indices_pool, axis=0)
        y_pool = np.delete(y_pool, training_indices_pool, axis=0)
        
    arquivo_performance_mlp.write("\n Avaliação_final_mlp \n")
    arquivo_performance_mlp.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_history_mlp, performance_history_mlp,delimiter=",")
    arquivo_performance_mlp.close()
    
def uncertain_sampling_xgb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from xgboost import XGBClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
           
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    # X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw)) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (bruto) de acordo com a dobra em uso
    X_treino, y_treino = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos para treinamento inicial
    n_labeled_examples = X_treino.shape[0]
    training_indices = np.random.randint(low=0, high=n_labeled_examples, size=5)
    
    X_train = X_treino[training_indices]
    y_train = y_treino[training_indices]
    
    #Isole os exemplos do pool para consultar
    
    X_pool = np.delete(X_treino, training_indices, axis=0)
    y_pool = np.delete(y_treino, training_indices, axis=0)
   
    #instanciando classificadores de aprendizado ativo
    learner_xgb = ActiveLearner(estimator=GradientBoostingClassifier(n_estimators=7, learning_rate=1.0,max_depth=1, random_state=42),X_training=X_train, y_training=y_train)
    arquivo_performance_xgb = open("random_performance_xgb_dobra_"+indica_pool+".txt","a")
    arquivo_history_xgb = ("random_history_xgb_dobra_"+indica_pool+".csv")
        
    for index in range(N_QUERIES):
        n_labeled_examples_pool = X_pool.shape[0]
        training_indices_pool = np.random.randint(low=0, high=n_labeled_examples_pool, size=BATCH_SIZE)
        X_temp= X_pool[training_indices_pool]
        y_temp= y_pool[training_indices_pool]
        X_train=np.append(X_train,X_temp, axis=0)
        y_train=np.append(y_train,y_temp, axis=0)
        learner_xgb.fit(X_train, y_train)
        predictions = learner_xgb.predict(X_teste)
        clear_output(wait=True)
        performance_history_xgb.append(accuracy_score(y_teste, predictions))
        print ('Accuracy_xgb after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
        arquivo_performance_xgb.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_performance_xgb.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_performance_xgb.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_performance.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_performance_xgb.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
        #print ("========================================")
        arquivo_performance_xgb.write('======================================== \n')
        #apaga registros consultados
        X_pool = np.delete(X_pool, training_indices_pool, axis=0)
        y_pool = np.delete(y_pool, training_indices_pool, axis=0)
        
    arquivo_performance_xgb.write("\n Avaliação_final_xgb \n")
    arquivo_performance_xgb.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_history_xgb, performance_history_xgb,delimiter=",")
    arquivo_performance_xgb.close()

def uncertain_sampling_svm(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn import svm
    from sklearn.tree import DecisionTreeClassifier
    from xgboost import XGBClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
           
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    # X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw)) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (bruto) de acordo com a dobra em uso
    X_treino, y_treino = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos para treinamento inicial
    n_labeled_examples = X_treino.shape[0]
    training_indices = np.random.randint(low=0, high=n_labeled_examples, size=5)
    
    X_train = X_treino[training_indices]
    y_train = y_treino[training_indices]
    
    #Isole os exemplos do pool para consultar
    
    X_pool = np.delete(X_treino, training_indices, axis=0)
    y_pool = np.delete(y_treino, training_indices, axis=0)
   
    #instanciando classificadores de aprendizado ativo
    #learner_svm = svm.LinearSVC()
    learner_svm = ActiveLearner(estimator=svm.SVC(kernel='linear',probability=True),X_training=X_train, y_training=y_train)
    arquivo_performance_svm = open("random_performance_svm_dobra_"+indica_pool+".txt","a")
    arquivo_history_svm = ("random_history_svm_dobra_"+indica_pool+".csv")
        
    for index in range(N_QUERIES):
        n_labeled_examples_pool = X_pool.shape[0]
        training_indices_pool = np.random.randint(low=0, high=n_labeled_examples, size=BATCH_SIZE)
        X_temp= X_pool[training_indices_pool]
        y_temp= y_pool[training_indices_pool]
        X_train=np.append(X_train,X_temp, axis=0)
        y_train=np.append(y_train,y_temp, axis=0)
        learner_svm.fit(X_train, y_train)
        predictions = learner_svm.predict(X_teste)
        clear_output(wait=True)
        performance_history_svm.append(accuracy_score(y_teste, predictions))
        print ('Accuracy_svm after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
        arquivo_performance_svm.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_performance_svm.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_performance_svm.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_performance.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_performance_svm.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
        #print ("========================================")
        arquivo_performance_svm.write('======================================== \n')
        #apaga registros consultados
        X_pool = np.delete(X_pool, training_indices_pool, axis=0)
        y_pool = np.delete(y_pool, training_indices_pool, axis=0)
        
    arquivo_performance_svm.write("\n Avaliação_final_svm \n")
    arquivo_performance_svm.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_history_svm, performance_history_svm,delimiter=",")
    arquivo_performance_svm.close()

def uncertain_sampling_nb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn import svm
    from sklearn.tree import DecisionTreeClassifier
    from xgboost import XGBClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
           
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    # X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw)) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (bruto) de acordo com a dobra em uso
    X_treino, y_treino = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos para treinamento inicial
    n_labeled_examples = X_treino.shape[0]
    training_indices = np.random.randint(low=0, high=n_labeled_examples, size=5)
    
    X_train = X_treino[training_indices]
    y_train = y_treino[training_indices]
    
    #Isole os exemplos do pool para consultar
    
    X_pool = np.delete(X_treino, training_indices, axis=0)
    y_pool = np.delete(y_treino, training_indices, axis=0)
   
    #instanciando classificadores de aprendizado ativo
    learner_nb = ActiveLearner(estimator=GaussianNB(),X_training=X_train, y_training=y_train)
    arquivo_performance_nb = open("random_performance_nb_dobra_"+indica_pool+".txt","a")
    arquivo_history_nb = ("random_history_nb_dobra_"+indica_pool+".csv")
        
    for index in range(N_QUERIES):
        n_labeled_examples_pool = X_pool.shape[0]
        training_indices_pool = np.random.randint(low=0, high=n_labeled_examples, size=BATCH_SIZE)
        X_temp= X_pool[training_indices_pool]
        y_temp= y_pool[training_indices_pool]
        X_train=np.append(X_train,X_temp, axis=0)
        y_train=np.append(y_train,y_temp, axis=0)
        learner_nb.fit(X_train, y_train)
        predictions = learner_nb.predict(X_teste)
        clear_output(wait=True)
        performance_history_nb.append(accuracy_score(y_teste, predictions))
        print ('Accuracy_nb after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
        arquivo_performance_nb.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_performance_nb.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_performance_nb.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_performance.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_performance_nb.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
        #print ("========================================")
        arquivo_performance_nb.write('======================================== \n')
        #apaga registros consultados
        X_pool = np.delete(X_pool, training_indices_pool, axis=0)
        y_pool = np.delete(y_pool, training_indices_pool, axis=0)
        
    arquivo_performance_nb.write("\n Avaliação_final_nb \n")
    arquivo_performance_nb.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_history_nb, performance_history_nb,delimiter=",")
    arquivo_performance_nb.close()    

import time
import sys
import threading
from datetime import datetime
from datetime import date

from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, train_test_split
from modAL.uncertainty import classifier_uncertainty
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from timeit import Timer
import time
import functools
# importing the multiprocessing module
import multiprocessing

#inicia relogio
t1 = time.time()
# Define o tamanho das divisões feitas no dataset (cross-validation)
n_dobras = 10
# Define Tamanho inicial da amostra (toda estratégia parte de um tamanho mínimo).
t_inicial = 10

#define array de indices das partições
idx_data =[]
# cross validation bags - n_splits
data_cv = StratifiedShuffleSplit(n_splits= n_dobras, random_state=42) 
data_cv.get_n_splits(X_raw, y_raw)
# chame a instância e gere os dados sobre a base original
type(data_cv.split(X_raw, y_raw))
# dividir os dados - A função split.split () retorna índices para amostras de treino e amostras de teste. 
# Ele examinará o número de validação cruzada especificado e retornará cada vez que treinar 
# e testar os índices de amostra usando os conjuntos de dados de treinamento e teste que podem 
# ser criados filtrando o conjunto de dados inteiro. Por exemplo idx_data[0][1], o primeiro indice faz referencia
# a dobra e o segundo indice faz referencia a posição da dobra (0 = treino e 1 = teste). Logo TRAIN=0 e TEST=1.
for train_index, test_index in data_cv.split(X_raw,y_raw):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("n_split",n_splits,"TRAIN:", train_index, "TEST:", test_index)
    idx_data.append([train_index, test_index])
#verifica tamanho das dobras (numero de instâncias de cada dobra)
#print("tamanho de cada dobra: ",idx_data[3][0].shape)

TRAIN =0
TEST =1

# DEFINE NUMERO DE QUERIES
BATCH_SIZE = 5
N_RAW_SAMPLES = 20
N_QUERIES = N_RAW_SAMPLES // BATCH_SIZE

#define arrays
performance_history_knn =[]
performance_history_rf =[]
performance_history_tree =[]
performance_history_mlp = []
performance_history_xgb = []
performance_history_svm = []
performance_history_nb = []

#chama procedimento de aprendizado para todas as dobras

for idx_dobra in range(n_dobras):    
    #threading.Thread(target=uncertain_sampling_knn(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    #threading.Thread(target=uncertain_sampling_rf(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    threading.Thread(target=uncertain_sampling_nb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    #threading.Thread(target=uncertain_sampling_mlp(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    #threading.Thread(target=uncertain_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    #threading.Thread(target=uncertain_sampling_xgb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    #threading.Thread(target=uncertain_sampling_svm(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()

t2 = time.time()
time_elapsed = (t2-t1)
hours, rem = divmod(time_elapsed, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))


In [ ]:
# Plot our performance over time.
fig, ax = plt.subplots(figsize=(8.5, 6), dpi=130)

ax.plot(performance_history_knn,color="blue")
ax.plot(performance_history_rf,color="red")
ax.plot(performance_history_tree,color="green")
#ax.plot(performance_history_mlp,color="yellow")
ax.plot(performance_history_xgb,color="orange")
ax.plot(performance_history_svm,color="brown")
ax.plot(performance_history_nb,color="pink")
ax.scatter(range(len(performance_history_knn)), performance_history_knn,s=0)
ax.scatter(range(len(performance_history_rf)), performance_history_rf, s=0)
ax.scatter(range(len(performance_history_tree)), performance_history_tree,s=0)
#ax.scatter(range(len(performance_history_mlp)), performance_history_mlp,s=0)
ax.scatter(range(len(performance_history_xgb)), performance_history_xgb,s=0)
ax.scatter(range(len(performance_history_svm)), performance_history_svm,s=0)
ax.scatter(range(len(performance_history_nb)), performance_history_nb,s=0)

ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=5, integer=True))
ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=10))
ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))

ax.set_ylim(bottom=0, top=1)
ax.grid(True)

ax.set_title('Incremental classification accuracy')
ax.set_xlabel('Query iteration')
ax.set_ylabel('Classification Accuracy')

plt.show()